# First-Steps EDA on `MLA_100k_checked_v3`

This notebook performs a **quick structural sanity check** of the dataset prior to feature engineering and modeling.

### Goals ⚽️
- Detect **sparse fields** (mostly empty / null).
- Flag **high-cardinality ID-like fields** (nearly all unique values).
- Detect **highly-uniform fields**

### What’s included 🧳
- Dataset load and schema preview.
- Missing-value summary by column.
- Cardinality analysis (unique ratios, potential identifiers).
- Basic type inference (numeric, categorical, text).
- Early notes on columns to **drop**

### Outputs 🎣
- Shortlist of columns to exclude (ID-like / unusable).
> These first checks help avoid data leakage (via hidden IDs) and reduce noise from unusable fields before running deeper EDA or modeling.

In [1]:
import pandas as pd
from challenge.new_or_used import build_dataset

## Load the dataset

In [2]:
X_train, y_train, X_test, y_test = build_dataset()
print(X_train[0], y_train[0])


{'seller_address': {'country': {'name': 'Argentina', 'id': 'AR'}, 'state': {'name': 'Capital Federal', 'id': 'AR-C'}, 'city': {'name': 'San Cristóbal', 'id': 'TUxBQlNBTjkwNTZa'}}, 'warranty': None, 'sub_status': [], 'deal_ids': [], 'base_price': 80.0, 'shipping': {'local_pick_up': True, 'methods': [], 'tags': [], 'free_shipping': False, 'mode': 'not_specified', 'dimensions': None}, 'non_mercado_pago_payment_methods': [{'description': 'Transferencia bancaria', 'id': 'MLATB', 'type': 'G'}, {'description': 'Acordar con el comprador', 'id': 'MLAWC', 'type': 'G'}, {'description': 'Efectivo', 'id': 'MLAMO', 'type': 'G'}], 'seller_id': 8208882349, 'variations': [], 'site_id': 'MLA', 'listing_type_id': 'bronze', 'price': 80.0, 'attributes': [], 'buying_mode': 'buy_it_now', 'tags': ['dragged_bids_and_visits'], 'listing_source': '', 'parent_item_id': 'MLA6553902747', 'coverage_areas': [], 'category_id': 'MLA126406', 'descriptions': ["{'id': 'MLA4695330653-912855983'}"], 'last_updated': '2015-09-

In [3]:
X_dataframe = pd.DataFrame(X_train)
X_dataframe.head()

,seller_address,warranty,sub_status,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,variations,site_id,...,stop_time,status,video_id,catalog_product_id,subtitle,initial_quantity,start_time,permalink,sold_quantity,available_quantity
0,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8208882349,[],MLA,...,1446669773000,active,None,NaN,None,1,1441485773000,http://articulo.mercadolibre.com.ar/MLA4695330...,0,1
1,"{'country': {'name': 'Argentina', 'id': 'AR'},...",NUESTRA REPUTACION,[],[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8141699488,[],MLA,...,1448474910000,active,None,NaN,None,1,1443290910000,http://articulo.mercadolibre.com.ar/MLA7160447...,0,1
2,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8386096505,[],MLA,...,1447027027000,active,None,NaN,None,1,1441843027000,http://articulo.mercadolibre.com.ar/MLA7367189...,0,1
3,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",5377752182,[],MLA,...,1449191596000,active,None,NaN,None,1,1443466076000,http://articulo.mercadolibre.com.ar/MLA9191625...,0,1
4,"{'country': {'name': 'Argentina', 'id': 'AR'},...",MI REPUTACION.,[],[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",2938071313,[],MLA,...,1445638040000,active,None,NaN,None,1,1440454040000,http://articulo.mercadolibre.com.ar/MLA7787961...,0,1


In [4]:
# I want to add the "used" column to the analysis
X_dataframe["used"] = [i == "used" for i in y_train]
X_dataframe["used"] = X_dataframe["used"].astype(int)

In [4]:
print(len(X_dataframe.columns))
X_dataframe.columns

44


Index(['seller_address', 'warranty', 'sub_status', 'deal_ids', 'base_price',
       'shipping', 'non_mercado_pago_payment_methods', 'seller_id',
       'variations', 'site_id', 'listing_type_id', 'price', 'attributes',
       'buying_mode', 'tags', 'listing_source', 'parent_item_id',
       'coverage_areas', 'category_id', 'descriptions', 'last_updated',
       'international_delivery_mode', 'pictures', 'id', 'official_store_id',
       'differential_pricing', 'accepts_mercadopago', 'original_price',
       'currency_id', 'thumbnail', 'title', 'automatic_relist', 'date_created',
       'secure_thumbnail', 'stop_time', 'status', 'video_id',
       'catalog_product_id', 'subtitle', 'initial_quantity', 'start_time',
       'permalink', 'sold_quantity', 'available_quantity'],
      dtype='object')

## Sparse Fields 🫙

In [ ]:
# Let's check if there are columns in which all the values have are empty or null
unuseful_columns = []

for col in X_dataframe.columns:
    # if it's a dictionary, is useful for the time being
    if isinstance(X_dataframe[col].iloc[0], dict):
        continue
    # if it's a list, check if all of them the same
    elif isinstance(X_dataframe[col].iloc[0], list):
        if X_dataframe[col].iloc[0] == []:
            if len(X_dataframe[col].value_counts()) == 1:
                unuseful_columns.append(col)
    else:
        if X_dataframe[col].nunique() == 1 or X_dataframe[col].nunique() == 0:
            unuseful_columns.append(col)

In [6]:
unuseful_columns

['site_id',
 'listing_source',
 'coverage_areas',
 'international_delivery_mode',
 'differential_pricing',
 'subtitle']

In [7]:
X_dataframe.drop(unuseful_columns, axis=1, inplace=True)
X_dataframe.head()

,seller_address,warranty,sub_status,deal_ids,base_price,shipping,non_mercado_pago_payment_methods,seller_id,variations,listing_type_id,...,secure_thumbnail,stop_time,status,video_id,catalog_product_id,initial_quantity,start_time,permalink,sold_quantity,available_quantity
0,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],80.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8208882349,[],bronze,...,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1446669773000,active,None,NaN,1,1441485773000,http://articulo.mercadolibre.com.ar/MLA4695330...,0,1
1,"{'country': {'name': 'Argentina', 'id': 'AR'},...",NUESTRA REPUTACION,[],[],2650.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8141699488,[],silver,...,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1448474910000,active,None,NaN,1,1443290910000,http://articulo.mercadolibre.com.ar/MLA7160447...,0,1
2,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],60.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",8386096505,[],bronze,...,https://a248.e.akamai.net/mla-s1-p.mlstatic.co...,1447027027000,active,None,NaN,1,1441843027000,http://articulo.mercadolibre.com.ar/MLA7367189...,0,1
3,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,[],[],580.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",5377752182,[],silver,...,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1449191596000,active,None,NaN,1,1443466076000,http://articulo.mercadolibre.com.ar/MLA9191625...,0,1
4,"{'country': {'name': 'Argentina', 'id': 'AR'},...",MI REPUTACION.,[],[],30.0,"{'local_pick_up': True, 'methods': [], 'tags':...","[{'description': 'Transferencia bancaria', 'id...",2938071313,[],bronze,...,https://a248.e.akamai.net/mla-s2-p.mlstatic.co...,1445638040000,active,None,NaN,1,1440454040000,http://articulo.mercadolibre.com.ar/MLA7787961...,0,1


In [ ]:
# check for >98% of the data null
null_percentage = X_dataframe.isnull().sum() / len(X_dataframe)
null_percentage = null_percentage[null_percentage > 0.98]
null_percentage


official_store_id     0.991722
original_price        0.998556
catalog_product_id    0.999922
dtype: float64

In [17]:
X_dataframe.drop(columns=list(null_percentage.index), inplace=True)

## High Cardinality ID-Like Fiels 🪪

In [ ]:
def all_values_different(series):
    """
    Check if all values in a pandas series are different.
    Handles different data types safely.
    Skips int and float columns.
    """
    try:
        if series.dtype in ['int64', 'float64']:
            return None  # Skip these columns
        
        if series.dtype == 'object':
            sample_values = series.dropna().head(10)
            if all(isinstance(val, (str, bool, type(None))) for val in sample_values):
                unique_count = series.nunique()
                return unique_count == len(series)
        
        string_series = series.astype(str)
        unique_count = string_series.nunique()
        return unique_count == len(series)
        
    except Exception as e:
        print(f"Error processing column {series.name}: {e}")
        return False

total_rows = len(X_dataframe)
all_different_columns = []
skipped_columns = []

print(f"Checking if all values are different in each column:")
print(f"Total rows: {total_rows}")
print("-" * 50)

for col in X_dataframe.columns:
    result = all_values_different(X_dataframe[col])
    
    if result is None:
        skipped_columns.append(col)
        print(f"⏭️  {col}: SKIPPED (int/float column)")
    elif result:
        all_different_columns.append(col)
        print(f"✅ {col}: ALL VALUES ARE DIFFERENT")
    else:
        print(f"❌ {col}: Has duplicate values")


Checking if all values are different in each column:
Total rows: 90000
--------------------------------------------------
❌ seller_address: Has duplicate values
❌ warranty: Has duplicate values
❌ sub_status: Has duplicate values
❌ deal_ids: Has duplicate values
⏭️  base_price: SKIPPED (int/float column)
❌ shipping: Has duplicate values
❌ non_mercado_pago_payment_methods: Has duplicate values
⏭️  seller_id: SKIPPED (int/float column)
❌ variations: Has duplicate values
❌ listing_type_id: Has duplicate values
⏭️  price: SKIPPED (int/float column)
❌ attributes: Has duplicate values
❌ buying_mode: Has duplicate values
❌ tags: Has duplicate values
❌ parent_item_id: Has duplicate values
❌ category_id: Has duplicate values
❌ descriptions: Has duplicate values
❌ last_updated: Has duplicate values
❌ pictures: Has duplicate values
✅ id: ALL VALUES ARE DIFFERENT
⏭️  official_store_id: SKIPPED (int/float column)
❌ accepts_mercadopago: Has duplicate values
⏭️  original_price: SKIPPED (int/float colu

In [10]:
# Summary of columns with all different values
print("\nSUMMARY:")
print("=" * 30)
if all_different_columns:
    print(f"Found {len(all_different_columns)} columns where ALL VALUES ARE DIFFERENT:")
    for col in all_different_columns:
        print(f"  - {col}")
else:
    print("No columns found where all values are different!")

if skipped_columns:
    print(f"\nSkipped {len(skipped_columns)} int/float columns:")
    for col in skipped_columns:
        print(f"  - {col}")

print(f"\nTotal columns: {len(X_dataframe.columns)}")
print(f"Analyzed columns: {len(X_dataframe.columns) - len(skipped_columns)}")
print(f"Skipped columns: {len(skipped_columns)}")
print(f"Columns with all different values: {len(all_different_columns)}")
print(f"Columns with some duplicate values: {len(X_dataframe.columns) - len(skipped_columns) - len(all_different_columns)}")



SUMMARY:
Found 2 columns where ALL VALUES ARE DIFFERENT:
  - id
  - permalink

Skipped 11 int/float columns:
  - base_price
  - seller_id
  - price
  - official_store_id
  - original_price
  - stop_time
  - catalog_product_id
  - initial_quantity
  - start_time
  - sold_quantity
  - available_quantity

Total columns: 38
Analyzed columns: 27
Skipped columns: 11
Columns with all different values: 2
Columns with some duplicate values: 25


In [11]:
if all_different_columns:
    print(f"Removing {len(all_different_columns)} columns where all values are different...")
    X_dataframe = X_dataframe.drop(columns=all_different_columns)
    print(f"Remaining columns: {len(X_dataframe.columns)}")
else:
    print("No columns to remove.")

# For now, let's just show what would be removed
if all_different_columns:
    print("Columns where all values are different (likely identifiers):")
    for col in all_different_columns:
        print(f"  - {col}")
else:
    print("No columns found where all values are different.")


Removing 2 columns where all values are different...
Remaining columns: 36
Columns where all values are different (likely identifiers):
  - id
  - permalink


## Highly Uniform Fiedls 🧑🏻‍✈️

In [12]:
# Check for columns where more than 95% of the data is the same value
def check_highly_uniform_columns(series, threshold=0.95):
    """
    Check if more than threshold% of values in a column are the same.
    Skips int and float columns.
    """
    try:
        # Skip int and float columns
        if series.dtype in ['int64', 'float64']:
            return None, None, None
        
        # For object columns, check if they contain basic types
        if series.dtype == 'object':
            sample_values = series.dropna().head(10)
            if all(isinstance(val, (str, bool, type(None))) for val in sample_values):
                value_counts = series.value_counts()
                if len(value_counts) > 0:
                    most_common_count = value_counts.iloc[0]
                    total_count = len(series)
                    percentage = most_common_count / total_count
                    most_common_value = value_counts.index[0]
                    return percentage, most_common_value, most_common_count
        
        # For complex types (dicts, lists), convert to string representation
        string_series = series.astype(str)
        value_counts = string_series.value_counts()
        if len(value_counts) > 0:
            most_common_count = value_counts.iloc[0]
            total_count = len(string_series)
            percentage = most_common_count / total_count
            most_common_value = value_counts.index[0]
            return percentage, most_common_value, most_common_count
        
        return 0, None, 0
        
    except Exception as e:
        print(f"Error processing column {series.name}: {e}")
        return 0, None, 0

# Check for highly uniform columns
uniform_columns = []
skipped_uniform_columns = []
threshold = 0.98

print(f"\nChecking for columns where >{threshold*100}% of data is the same value:")
print(f"Total rows: {total_rows}")
print("-" * 60)

for col in X_dataframe.columns:
    percentage, most_common_value, count = check_highly_uniform_columns(X_dataframe[col], threshold)
    
    if percentage is None:
        skipped_uniform_columns.append(col)
        print(f"⏭️  {col}: SKIPPED (int/float column)")
    elif percentage > threshold:
        uniform_columns.append((col, percentage, most_common_value, count))
        print(f"🔴 {col}: {percentage*100:.1f}% same value")
        print(f"   Most common: '{most_common_value}' (appears {count} times)")
    else:
        print(f"✅ {col}: {percentage*100:.1f}% most common value (below threshold)")



Checking for columns where >98.0% of data is the same value:
Total rows: 90000
------------------------------------------------------------
✅ seller_address: 4.1% most common value (below threshold)
✅ warranty: 5.7% most common value (below threshold)
🔴 sub_status: 99.0% same value
   Most common: '[]' (appears 89109 times)
🔴 deal_ids: 99.8% same value
   Most common: '[]' (appears 89783 times)
⏭️  base_price: SKIPPED (int/float column)
✅ shipping: 41.4% most common value (below threshold)
✅ non_mercado_pago_payment_methods: 30.6% most common value (below threshold)
⏭️  seller_id: SKIPPED (int/float column)
✅ variations: 91.8% most common value (below threshold)
✅ listing_type_id: 63.2% most common value (below threshold)
⏭️  price: SKIPPED (int/float column)
✅ attributes: 87.6% most common value (below threshold)
✅ buying_mode: 97.0% most common value (below threshold)
✅ tags: 72.6% most common value (below threshold)
✅ parent_item_id: 0.0% most common value (below threshold)
✅ categ

In [13]:
# Summary of highly uniform columns
print("\nUNIFORM COLUMNS SUMMARY:")
print("=" * 40)
if uniform_columns:
    print(f"Found {len(uniform_columns)} columns where >{threshold*100}% of data is the same:")
    for col, percentage, most_common_value, count in uniform_columns:
        print(f"  - {col}: {percentage*100:.1f}% are '{most_common_value}' ({count} times)")
    print(f"\nThese columns might be:")
    print(f"  • Low-variance features (not very informative)")
    print(f"  • Consider removing or feature engineering")
else:
    print(f"No columns found where >{threshold*100}% of data is the same!")

print(f"\nTotal columns analyzed: {len(X_dataframe.columns) - len(skipped_uniform_columns)}")
print(f"Highly uniform columns: {len(uniform_columns)}")
print(f"Columns with good variance: {len(X_dataframe.columns) - len(skipped_uniform_columns) - len(uniform_columns)}")



UNIFORM COLUMNS SUMMARY:
Found 3 columns where >98.0% of data is the same:
  - sub_status: 99.0% are '[]' (89109 times)
  - deal_ids: 99.8% are '[]' (89783 times)
  - currency_id: 99.4% are 'ARS' (89496 times)

These columns might be:
  • Low-variance features (not very informative)
  • Consider removing or feature engineering

Total columns analyzed: 25
Highly uniform columns: 3
Columns with good variance: 22


In [14]:
if uniform_columns:
    columns_to_remove = [col for col, _, _, _ in uniform_columns]
    print(f"Removing {len(columns_to_remove)} highly uniform columns...")
    X_dataframe = X_dataframe.drop(columns=columns_to_remove)
    print(f"Remaining columns: {len(X_dataframe.columns)}")
else:
    print("No highly uniform columns to remove.")

Removing 3 highly uniform columns...
Remaining columns: 33


# Save the dataframe 💾

In [18]:
# Add the label and save the dataframe
X_dataframe["used"] = [y =="used" for y in y_train]
X_dataframe.to_csv("../data/X_dataframe.csv", index=False)